# GSTORE CUSTOMER REVENUE PREDICTION - DATA CLEANING and MODEL BUILDING
*By Harish Julapalli*

Importing required libraries:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import json


%matplotlib inline
plt.style.use('ggplot')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Data Loading:

In [ ]:
def load_df(csv_path='../input/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df
        

In [ ]:
train_df = load_df(nrows = 200000)

In [ ]:
train_df.head()

In [ ]:
test_df = load_df(csv_path='../input/test_v2.csv', nrows=50000)

In [ ]:
train_df.info()

Removing single category columns:

In [ ]:
single_cat_cols_train = [col for col in train_df.columns if train_df.nunique()[col] == 1]
train_df.drop(single_cat_cols_train, axis = 1, inplace = True)

In [ ]:
single_cat_cols_test = [col for col in test_df.columns if test_df.nunique()[col] == 1]
test_df.drop(single_cat_cols_test, axis = 1, inplace = True)

In [ ]:
list(set(test_df.columns) - set(train_df.columns))

Removing columns: 'customDimensions','hits','trafficSource_adwordsClickInfo.page', as they don't add any meaning.

In [ ]:
train_df.drop(['customDimensions','hits','trafficSource_adwordsClickInfo.page'], axis = 1, inplace = True)
test_df.drop(['customDimensions','hits'], axis = 1, inplace = True)

In [ ]:
train_df.info()

Converting numeric columns from 'Object' to 'float':

In [ ]:
numeric_cols = [col for col in train_df.columns if 'totals_' in col]
print(numeric_cols)

In [ ]:
train_df[numeric_cols] = train_df[numeric_cols].astype('float64')
test_df[numeric_cols] = test_df[numeric_cols].astype('float64')

**visitStartTime** column has UNIX time, which gives us time attribute to. So, retaining it and removing 'Date' column.

In [ ]:
train_df['visitStartTime'] = pd.to_datetime(train_df['visitStartTime'], unit = 's')
test_df['visitStartTime'] = pd.to_datetime(test_df['visitStartTime'], unit = 's')

In [ ]:
train_df.drop('date', axis = 1, inplace = True)
test_df.drop('date', axis = 1, inplace = True)

In [ ]:
train_df['visitId'] = train_df['visitId'].astype('object')
test_df['visitId'] = test_df['visitId'].astype('object')

In [ ]:
train_df.info()

Extracting Time features:

In [ ]:
train_df = train_df.set_index('visitStartTime')

In [ ]:
train_df['day'] = train_df.index.day
train_df['month'] = train_df.index.month
train_df['day_of_week'] = train_df.index.dayofweek
train_df['hour'] = train_df.index.hour

In [ ]:
test_df = test_df.set_index('visitStartTime')
test_df['day'] = test_df.index.day
test_df['month'] = test_df.index.month
test_df['day_of_week'] = test_df.index.dayofweek
test_df['hour'] = test_df.index.hour

In [ ]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()
train_df.head()

Treating Missing values:

Percantage of missing values per each column:

In [ ]:
(train_df.isna().sum()/train_df.shape[0])*100

Filling missing values in numeric columns with '0':

In [ ]:
numeric_cols

In [ ]:
train_df[numeric_cols] = train_df[numeric_cols].fillna(0)

In [ ]:
test_df[numeric_cols] = test_df[numeric_cols].fillna(0)

Although categorical columns seesms to have no missing values, there are many rows with values like, **(not set)** and **not available in demo dataset** to **unknown**.

In [ ]:
cat_cols = [col for col in train_df.columns if train_df[col].dtype == 'object']
cat_cols.remove('fullVisitorId')
cat_cols.remove('visitId')
print(cat_cols)

In [ ]:
for col in cat_cols:
    train_df[col] = train_df[col].str.replace('^\(.*', 'unknown', regex = True)
    train_df[col] = train_df[col].str.replace('not available in demo dataset', 'unknown', regex = False)

In [ ]:
for col in cat_cols:
    test_df[col] = test_df[col].str.replace('^\(.*', 'unknown', regex = True)
    test_df[col] = test_df[col].str.replace('not available in demo dataset', 'unknown', regex = False)

In [ ]:
(train_df.isna().sum()/train_df.shape[0])*100

In [ ]:
train_df.drop('trafficSource_adwordsClickInfo.gclId', axis = 1, inplace = True)
test_df.drop('trafficSource_adwordsClickInfo.gclId', axis = 1, inplace = True)

In [ ]:
traffic_source_cols = [col for col in train_df.columns if 'trafficSource_' in col]
traffic_source_cols.remove('trafficSource_campaign')
traffic_source_cols.remove('trafficSource_medium')
traffic_source_cols

In [ ]:
for col in traffic_source_cols:
    train_df[col].fillna('unknown', inplace = True)
    test_df[col].fillna('unknown', inplace = True)

In [ ]:
def add_new_category(x):
    x = str(x).lower()
    if 'google' in x:
        return 'google'
    elif 'baidu' in x:
        return 'baidu'
    elif 'facebook' in x:
        return 'facebook'
    elif 'reddit' in x:
        return 'reddit'
    elif 'yahoo' in x:
        return 'yahoo'
    elif 'bing' in x:
        return 'bing'
    elif 'yandex' in x:
        return 'yandex'
    else:
        return 'other'

In [ ]:
train_df['trafficSource_source'] = train_df['trafficSource_source'].apply(add_new_category)
test_df['trafficSource_source'] = test_df['trafficSource_source'].apply(add_new_category)

In [ ]:
def add_new_category(x):
    x = str(x).lower()
    if x == 'nan':
        return 'nan'
    
    x = ''.join(x.split())
    
    if 'youtube' in x or 'you' in x or 'yo' in x or 'tub' in x:
        return 'youtube'
    elif 'google' in x or 'goo' in x or 'gle' in x:
        return 'google'
    else:
        return 'other'

In [ ]:
train_df['trafficSource_keyword'] = train_df['trafficSource_keyword'].apply(add_new_category)
test_df['trafficSource_keyword'] = test_df['trafficSource_keyword'].apply(add_new_category)

In [ ]:
(train_df.isna().sum()/train_df.shape[0])*100

In [ ]:
train_df['mean_hits_per_networkDomain'] = train_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('mean').astype('int')
train_df['mean_pageViews_per_networkDomain'] = train_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('mean').astype('int')

In [ ]:
test_df['mean_hits_per_networkDomain'] = test_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('mean').astype('int')
test_df['mean_pageViews_per_networkDomain'] = test_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('mean').astype('int')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_fullVisitorId = train_df['fullVisitorId']
train_visitId = train_df['visitId']
train_date = train_df['visitStartTime']

test_fullVisitorId = test_df['fullVisitorId']
test_visitId = test_df['visitId']
test_date = test_df['visitStartTime']

train_df.drop(['fullVisitorId', 'visitId','visitStartTime'], axis=1, inplace=True)
test_df.drop(['fullVisitorId', 'visitId','visitStartTime'], axis=1, inplace=True)

In [ ]:
train_df.head()

In [ ]:
train_df[['totals_totalTransactionRevenue','totals_transactionRevenue']].corr()

In [ ]:
train_df.drop(['totals_totalTransactionRevenue'], axis=1, inplace=True)
test_df.drop(['totals_totalTransactionRevenue'], axis=1, inplace=True)

In [ ]:
cat_cols = [col for col in train_df.columns if train_df[col].dtype in ['object','bool']]

Encoding categorical columns:

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

train_df[cat_cols] = train_df[cat_cols].apply(lb.fit_transform)
test_df[cat_cols] = test_df[cat_cols].apply(lb.fit_transform)

In [ ]:
train_df.head()

Separating training examples and target variables:

In [ ]:
train_X = train_df.drop('totals_transactionRevenue', axis = 1)
train_y = train_df['totals_transactionRevenue'].apply(np.log1p)

In [ ]:
test_X = test_df.drop('totals_transactionRevenue', axis = 1)
test_y = test_df['totals_transactionRevenue'].apply(np.log1p)

In [ ]:
dev_X = train_X.loc[0:150000,:]
dev_y = train_y.loc[0:150000]

val_X = train_X.loc[150000:,:]
val_y = train_y.loc[150000:]

**MODEL BUILDING**

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
xg_reg = xgb.XGBRegressor(objective = 'reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10,
                         n_estimators = 10)

In [ ]:
xg_reg.fit(dev_X, dev_y)

In [ ]:
preds = xg_reg.predict(val_X)

In [ ]:
rmse = np.sqrt(mean_squared_error(val_y, preds))
print(rmse)

In [ ]:
val_fullVisitorId = train_fullVisitorId[150000:]

In [ ]:
val_pred_df = pd.DataFrame({'fullVisitorID':val_fullVisitorId.values})
val_pred_df["transactionRevenue"] = val_y.values
val_pred_df["PredictedRevenue"] = preds

In [ ]:
val_pred_df = val_pred_df.groupby("fullVisitorID")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(mean_squared_error(val_pred_df["transactionRevenue"].values, val_pred_df["PredictedRevenue"].values)))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
params = {'learning_rate':[0.01, 0.1, 0.2], 'max_depth':[3,5,7,9], 'colsample_bytree':[0.5,0.7,1]} 

In [ ]:
model = xgb.XGBRegressor()

In [ ]:
grid = GridSearchCV(estimator = model, param_grid = params)

In [ ]:
grid.fit(dev_X, dev_y)

In [ ]:
grid.best_params_

In [ ]:
xg_reg_grid = xgb.XGBRegressor(objective = 'reg:linear', colsample_bytree = 0.7, learning_rate = 0.1, max_depth = 3, alpha = 10,
                         n_estimators = 10)

In [ ]:
xg_reg_grid.fit(dev_X, dev_y)

In [ ]:
preds_grid = xg_reg_grid.predict(val_X)

In [ ]:
rmse = np.sqrt(mean_squared_error(val_y, preds_grid))
print(rmse)